In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd

In [26]:
# Define the file paths
file1_path = "/content/drive/My Drive/data/train_data.csv"
file2_path = "/content/drive/My Drive/data/test_data.csv"
file3_path = "/content/drive/My Drive/data/train_labels.csv"

# # Load the data into Pandas DataFrames
# train_data = pd.read_csv(file1_path)
# test_data = pd.read_csv(file2_path)
# train_labels = pd.read_csv(file3_path)


In [79]:
train_data = pd.read_csv("/content/drive/My Drive/data/train_data.csv", header=None)
train_labels = pd.read_csv("/content/drive/My Drive/data/train_labels.csv", header=None)
test_data = pd.read_csv("/content/drive/My Drive/data/test_data.csv", header=None)

print('train_data.shape: {}'.format(train_data.shape))
print('train_labels.shape: {}'.format(train_labels.shape))
print('test_data.shape: {}'.format(test_data.shape))


train_data.shape: (160, 22036)
train_labels.shape: (160, 1)
test_data.shape: (40, 22036)


In [80]:
# imports:

from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


In [81]:
X = train_data
y = train_labels

print("X shape: ", X.shape)
print("y shape: ", y.shape)

X shape:  (160, 22036)
y shape:  (160, 1)


In [35]:
# Train/Test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)

In [36]:
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_val.shape)
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_val.shape)

X_train shape:  (128, 22036)
X_test shape:  (32, 22036)
y_train shape:  (128, 1)
y_test shape:  (32, 1)


In [83]:
# PCA for Dimensionality Reduction
# Define the PCA and OneVsRestClassifier

# Instantiating

pca = PCA()
xgb_model = XGBClassifier(objective='multi:softprob')


In [43]:
# Define PCA and XGBoost models already fine tuned:


pca = PCA(n_components=50)
xgb_model = XGBClassifier(objective='multi:softmax',
                          num_class=5,  # Number of classes in your multi-class classification problem
                          learning_rate=0.03,
                          max_depth=3,
                          min_child_weight=3,
                          n_estimators=150,
                          gamma=0.0,
                          reg_alpha=0.01,
                          n_jobs=4
                          )

# Define the pipeline including PCA and XGBoost
pipeline_02 = Pipeline([
    ('pca', pca),
    ('xgb', xgb_model)
])



In [55]:
# Define the OneVsRestClassifier
ovr_classifier = OneVsRestClassifier(pipeline_02)

# Fit the classifier to your training data
ovr_classifier.fit(train_data, train_labels)

OneVsRestClassifier(estimator=Pipeline(steps=[('pca', PCA(n_components=50)),
                                              ('xgb',
                                               XGBClassifier(base_score=None,
                                                             booster=None,
                                                             callbacks=None,
                                                             colsample_bylevel=None,
                                                             colsample_bynode=None,
                                                             colsample_bytree=None,
                                                             device=None,
                                                             early_stopping_rounds=None,
                                                             enable_categorical=False,
                                                             eval_metric=None,
                                                             feature_types=None,
                                                             gamma=0.0,
                                                             grow_policy=None,
                                                             importance_type=None,
                                                             interaction_constraints=None,
                                                             learning_rate=0.03,
                                                             max_bin=None,
                                                             max_cat_threshold=None,
                                                             max_cat_to_onehot=None,
                                                             max_delta_step=None,
                                                             max_depth=3,
                                                             max_leaves=None,
                                                             min_child_weight=3,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=150,
                                                             n_jobs=4,
                                                             num_class=5,
                                                             num_parallel_tree=None, ...))]))

In [64]:
# Evaluate the best model on the validation set
val_predictions = ovr_classifier.predict(X_val)
val_accuracy = accuracy_score(y_val, val_predictions)
print("Validation accuracy: {:.2f}".format(val_accuracy))


Validation accuracy: 1.00


In [75]:
# Now we can use ovr_classifier.predict() to predict labels for the test data
predicted_labels = ovr_classifier.predict(test_data)
predicted_labels.shape

(40,)

In [87]:
test_accuracy = accuracy_score(test_data, predicted_labels)
print("Test accuracy:", test_accuracy)

ValueError: Classification metrics can't handle a mix of continuous-multioutput and multiclass targets